**I recommend you run the first code cell of this notebook immediately, to start provisioning drake on the cloud machine, then you can leave this window open as you [read the textbook](http://manipulation.csail.mit.edu/clutter.html).**

# Notebook Setup

The following cell will:
- on Colab (only), install Drake to `/opt/drake`, install Drake's prerequisites via `apt`, and add pydrake to `sys.path`.  This will take approximately two minutes on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  If you navigate between notebooks using Colab's "File->Open" menu, then you can avoid provisioning a separate machine for each notebook.
- define some utility methods/classes that will eventually disappear from this notebook and live in drake.

You will need to rerun this cell if you restart the kernel, but it should be fast because the machine will already have drake installed.

In [ ]:
import importlib
import sys
from urllib.request import urlretrieve

if 'google.colab' in sys.modules and importlib.util.find_spec('manipulation') is None:
    urlretrieve(f"http://manipulation.csail.mit.edu/scripts/setup/setup_manipulation_colab.py",
                "setup_manipulation_colab.py")
    from setup_manipulation_colab import setup_manipulation
    setup_manipulation(manipulation_sha='master', drake_version='latest', drake_build='continuous')

# Determine if this notebook is currently running as a notebook or a unit test.
from IPython import get_ipython
running_as_notebook = get_ipython() and hasattr(get_ipython(), 'kernel')

# Setup rendering (with xvfb), if necessary:
import os
if 'google.colab' in sys.modules and os.getenv("DISPLAY") is None:
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(1400, 900))
    display.start()

server_args = []
if 'google.colab' in sys.modules:
  server_args = ['--ngrok_http_tunnel']

# Start two meshcat servers (one for 2d) to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc_2d, zmq_url_2d, web_url_2d = start_zmq_server_as_subprocess(server_args=server_args)
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

# Imports
import numpy as np
from IPython.display import display, HTML
from ipywidgets import Textarea

from pydrake.all import ( 
    AddMultibodyPlantSceneGraph, ConnectMeshcatVisualizer, 
    DiagramBuilder, RigidTransform, RotationMatrix, Box,    
    CoulombFriction, FindResourceOrThrow, FixedOffsetFrame, 
    GeometryInstance, MeshcatContactVisualizer, Parser, PlanarJoint,  
    JointIndex, Simulator, ProcessModelDirectives, LoadModelDirectives
)

from functools import partial
import open3d as o3d
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import meshcat
import meshcat.geometry as g
import meshcat.transformations as tf

from pydrake.all import (
    ConnectPlanarSceneGraphVisualizer,
    ConnectDrakeVisualizer, DepthCameraProperties, RgbdSensor,
    RandomGenerator, UniformlyRandomRotationMatrix, RollPitchYaw,
    MakeRenderEngineVtk, RenderEngineVtkParams, Role, UnitInertia, set_log_level
)

from ipywidgets import Dropdown, FloatSlider, Layout
from pydrake.all import (
    Sphere, Cylinder, Box, Capsule, Ellipsoid, SpatialInertia)


#from pydrake.multibody.jupyter_widgets import MakeJointSlidersThatPublishOnCallback
from manipulation.jupyter_widgets import MakeJointSlidersThatPublishOnCallback
from manipulation.meshcat_utils import draw_open3d_point_cloud, draw_points
from manipulation.open3d_utils import create_open3d_point_cloud
from manipulation.mustard_depth_camera_example import MustardExampleSystem
from manipulation.scenarios import AddRgbdSensors
from manipulation.utils import FindResource

set_log_level("warn");

ycb = [("cracker", "003_cracker_box.sdf"), 
        ("sugar", "004_sugar_box.sdf"), 
        ("soup", "005_tomato_soup_can.sdf"), 
        ("mustard", "006_mustard_bottle.sdf"), 
        ("gelatin", "009_gelatin_box.sdf"), 
        ("meat", "010_potted_meat_can.sdf")]


# Falling things (in 2D)


In [ ]:
def clutter_gen():
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)

    box = Box(10., 10., 10.)
    X_WBox = RigidTransform([0, 0, -5])
    mu = 1.0
    plant.RegisterCollisionGeometry(plant.world_body(), X_WBox, box, "ground", CoulombFriction(mu, mu))
    plant.RegisterVisualGeometry(plant.world_body(), X_WBox, box, "ground", [.9, .9, .9, 1.0])
    planar_joint_frame = plant.AddFrame(FixedOffsetFrame("planar_joint_frame", plant.world_frame(), RigidTransform(RotationMatrix.MakeXRotation(np.pi/2))))

    parser = Parser(plant)

    sdf = FindResourceOrThrow("drake/examples/manipulation_station/models/061_foam_brick.sdf")

    for i in range(20 if running_as_notebook else 2):
        instance = parser.AddModelFromFile(sdf, f"object{i}")
        plant.AddJoint(PlanarJoint(f"joint{i}", planar_joint_frame, plant.GetFrameByName("base_link", instance), damping=[0,0,0]))

    plant.Finalize()

    vis = ConnectPlanarSceneGraphVisualizer(
        builder, 
        scene_graph,
        xlim=[-.6, .6],
        ylim=[-.1, 0.5],
        show=False,
    )

    diagram = builder.Build()
    simulator = Simulator(diagram)
    plant_context = plant.GetMyContextFromRoot(simulator.get_mutable_context())

    rs = np.random.RandomState()
    z = 0.1
    for i in range(plant.num_joints()):
        joint = plant.get_joint(JointIndex(i))
        joint.set_pose(plant_context, [rs.uniform(-.4, .4), z], rs.uniform(-np.pi/2.0, np.pi/2.0))
        z += 0.1

    vis.start_recording()
    simulator.AdvanceTo(1.5 if running_as_notebook else 0.1)
    vis.stop_recording()
    ani = vis.get_recording_as_animation(repeat=False)
    display(HTML(ani.to_jshtml()))
    
clutter_gen()

# Falling things (in 3D)


In [ ]:
def clutter_gen():
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0005)

    parser = Parser(plant)

    parser.AddModelFromFile(FindResourceOrThrow(
        "drake/examples/manipulation_station/models/bin.sdf"))
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("bin_base"))


    rs = np.random.RandomState()  # this is for python
    generator = RandomGenerator(rs.randint(1000))  # this is for c++
    for i in range(10 if running_as_notebook else 2):
        object_num = rs.randint(len(ycb))
        sdf = FindResourceOrThrow("drake/manipulation/models/ycb/sdf/" + ycb[object_num][1])
        parser.AddModelFromFile(sdf, f"object{i}")

    plant.Finalize()

    renderer = "my_renderer"
    scene_graph.AddRenderer(
        renderer, MakeRenderEngineVtk(RenderEngineVtkParams()))
    properties = DepthCameraProperties(width=640,
                                       height=480,
                                       fov_y=np.pi / 4.0,
                                       renderer_name=renderer,
                                       z_near=0.1,
                                       z_far=10.0)
    camera = builder.AddSystem(
        RgbdSensor(parent_id=scene_graph.world_frame_id(),
                   X_PB=RigidTransform(
                       RollPitchYaw(np.pi, 0, np.pi/2.0),
                       [0, 0, .8]),
                   properties=properties,
                   show_window=False))
    camera.set_name("rgbd_sensor")
    builder.Connect(scene_graph.get_query_output_port(),
                    camera.query_object_input_port())
    builder.ExportOutput(camera.color_image_output_port(), "color_image")

    # Note: if you're running this on a local machine, then you can 
    # use drake_visualizer to see the simulation.  (It's too slow to 
    # show the meshes on meshcat).
    vis = ConnectDrakeVisualizer(
        builder, 
        scene_graph
    )

    diagram = builder.Build()
    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()
    plant_context = plant.GetMyContextFromRoot(context)

    z = 0.1
    for body_index in plant.GetFloatingBaseBodies():
        tf = RigidTransform(
                UniformlyRandomRotationMatrix(generator),  
                [rs.uniform(-.15,.15), rs.uniform(-.2, .2), z])
        plant.SetFreeBodyPose(plant_context, 
                              plant.get_body(body_index),
                              tf)
        z += 0.1

    simulator.AdvanceTo(1.0 if running_as_notebook else 0.1)
    color_image = diagram.GetOutputPort("color_image").Eval(context)
    plt.figure()
    plt.imshow(color_image.data)
    plt.axis('off')

clutter_gen()

# Contact force "inspector"


In [ ]:
def contact_force_inspector(slope=0.0, mu=1.0, second_brick=False):
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.01)

    box = Box(10., 10., 10.)
    X_WBox = RigidTransform(RotationMatrix.MakeYRotation(slope), [0, 0, -5.05])
    plant.RegisterCollisionGeometry(plant.world_body(), X_WBox, box, "ground", CoulombFriction(mu, mu))
    plant.RegisterVisualGeometry(plant.world_body(), X_WBox, box, "ground", [.9, .9, .9, 1.0])

    parser = Parser(plant)
    brick_sdf = FindResource("models/planar_foam_brick_collision_as_visual.sdf")
    parser.AddModelFromFile(brick_sdf)
    frame = plant.AddFrame(FixedOffsetFrame("planar_joint_frame", plant.world_frame(), RigidTransform(RotationMatrix.MakeXRotation(np.pi/2))))
    plant.AddJoint(PlanarJoint("brick", frame, plant.GetFrameByName("base_link"), damping=[0,0,0]))

    if second_brick:
        brick2 = parser.AddModelFromFile(brick_sdf, "brick2")
        plant.AddJoint(PlanarJoint("brick2", frame, plant.GetFrameByName("base_link", brick2), damping=[0,0,0]))

    plant.Finalize()

    meshcat = ConnectMeshcatVisualizer(
        builder, 
        scene_graph, 
        zmq_url=zmq_url_2d,
        server_args=server_args)
    meshcat.set_planar_viewpoint(xmin=-.2, xmax=.2, ymin=-.2, ymax=0.3)

    contact_visualizer = builder.AddSystem(MeshcatContactVisualizer(meshcat, 
                                           force_threshold=0.0,
                                           contact_force_scale=1.0, 
                                           contact_force_radius=0.002,   
                                           plant=plant))
    builder.Connect(scene_graph.get_pose_bundle_output_port(),
                    contact_visualizer.GetInputPort("pose_bundle"))
    builder.Connect(plant.get_contact_results_output_port(),
                    contact_visualizer.GetInputPort("contact_results"))

    diagram = builder.Build()
    meshcat.load()
    context = diagram.CreateDefaultContext()

    textoutput = Textarea(value="", description="contact info: ", layout={'width':'800px','height':'100px'}, style={'description_width':'initial'})
    display(textoutput)
    def my_callback(plant_context):
        results = plant.get_contact_results_output_port().Eval(plant_context)
        textoutput.value = "" if results.num_point_pair_contacts()>0 else "no contact"
        for i in range(results.num_point_pair_contacts()):
            info = results.point_pair_contact_info(i)
            pair = info.point_pair()
            textoutput.value += f"slip speed:{info.slip_speed():.4f}, depth:{pair.depth:.4f}, "
            textoutput.value += "force:" + np.array2string(info.contact_force(), formatter={'float': lambda x: "{:5.2f}".format(x)}) + "\n"

    lower_limit = [-0.1, -0.1, -np.pi/2.0]
    upper_limit = [0.1, 0.1, np.pi/2.0]
    if second_brick:
        lower_limit += lower_limit
        upper_limit += upper_limit
        plant.SetPositions(
            plant.GetMyContextFromRoot(context),
            [0, 0, 0, 0.07, 0.07, 0.0])

    MakeJointSlidersThatPublishOnCallback(
        plant, diagram, context, my_callback, resolution=0.001,
        lower_limit=lower_limit, upper_limit=upper_limit)    

contact_force_inspector(
    slope=0.1, 
    mu=0.5,
    second_brick=True);

# Contact results "inspector"

This simple visualization shows some of the complexity of the contact geometry problem.  I will make it better, but right now, when you move the objects into contact of each other you will see three points:  the contact point is in **red**, the contact normal is added to the contact point with the tip as **green**, and the (scaled) contact force tip is drawn in **blue**.  Contact points on the bodies are drawn in **orange**.

In [ ]:
shapes = ["Point", "Sphere", "Cylinder", "Box", "Capsule", "Ellipsoid"]
shapeA = Dropdown(options=shapes,
                  description="Body A",
                  value="Box",
                  style = {'description_width': 'initial'})
shapeB = Dropdown(options=shapes,
                  description="Body B",
                  style = {'description_width': 'initial'})
display(shapeA)
display(shapeB)

def get_shape(shape):
    if shape == "Point":
        return Sphere(0.01)
    elif shape == "Sphere":
        return Sphere(1.0)
    elif shape == "Cylinder":
        return Cylinder(1.0, 2.0)
    elif shape == "Box":
        return Box(1.0, 2.0, 3.0)
    elif shape == "Capsule":
        return Capsule(1.0, 2.0)
    elif shape == "Ellipsoid":
        return Ellipsoid(1.0, 2.0, 3.0) 
    raise Exception("Unknown shape")


In [ ]:
def contact_inspector(shapeA, shapeB):
    builder = DiagramBuilder()

    shapeA = get_shape(shapeA.value)
    shapeB = get_shape(shapeB.value)

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
    frame = plant.AddFrame(FixedOffsetFrame("planar_joint_frame", plant.world_frame(), RigidTransform(RotationMatrix.MakeXRotation(np.pi/2))))

    mu = 0.5
    bodyA = plant.AddRigidBody("A", SpatialInertia(mass=1.0, p_PScm_E=np.array([0., 0., 0.]),
            G_SP_E=UnitInertia(1.0, 1.0, 1.0)))
    plant.RegisterCollisionGeometry(bodyA, RigidTransform(), shapeA, "A", CoulombFriction(mu, mu))
    plant.RegisterVisualGeometry(bodyA, RigidTransform(), shapeA, "A", [.9, .5, .5, 0.5])
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("A"))

    bodyB = plant.AddRigidBody("B", SpatialInertia(mass=1.0, p_PScm_E=np.array([0., 0., 0.]),
            G_SP_E=UnitInertia(1.0, 1.0, 1.0)))
    plant.RegisterCollisionGeometry(bodyB, RigidTransform(), shapeB, "B", CoulombFriction(mu, mu))
    plant.RegisterVisualGeometry(bodyB, RigidTransform(), shapeB, "B", [.5, .5, .9, 0.9])
    plant.AddJoint(PlanarJoint("B", frame, plant.GetFrameByName("B"), damping=[0,0,0]))

    plant.Finalize()

    meshcat = ConnectMeshcatVisualizer(
        builder, 
        scene_graph, 
        zmq_url=zmq_url_2d,
        server_args=server_args)
    meshcat.set_planar_viewpoint(xmin=-3.0, xmax=3.0, ymin=-.2, ymax=3.0)

#    contact_visualizer = builder.AddSystem(MeshcatContactVisualizer(meshcat, 
#                                           force_threshold=0.0,
#                                           contact_force_scale=1.0, 
#                                           contact_force_radius=0.002,   
#                                           plant=plant))
#    builder.Connect(scene_graph.get_pose_bundle_output_port(),
#                    contact_visualizer.GetInputPort("pose_bundle"))
#    builder.Connect(plant.get_contact_results_output_port(),
#                    contact_visualizer.GetInputPort("contact_results"))

    diagram = builder.Build()
    meshcat.load()
    context = diagram.CreateDefaultContext()

    contact = meshcat.vis["contact"]
    red = g.MeshLambertMaterial(color=0xff0000)
    green = g.MeshLambertMaterial(color=0x00ff00)
    blue = g.MeshLambertMaterial(color=0x0000ff)
    orange = g.MeshLambertMaterial(color=0xffa500)    

    textoutput = Textarea(value="", description="contact info: ", layout={'width':'800px','height':'100px'}, style={'description_width':'initial'})
    display(textoutput)
    def my_callback(plant_context):
        results = plant.get_contact_results_output_port().Eval(plant_context)
        contact.delete()
        textoutput.value = "" if results.num_point_pair_contacts()>0 else "no contact"
        for i in range(results.num_point_pair_contacts()):
            info = results.point_pair_contact_info(i)
            pair = info.point_pair()
            contact[str(i)].set_object(g.Sphere(0.02), red)
            contact[str(i)].set_transform(tf.translation_matrix(info.contact_point()))
            contact[str(i)+"A"].set_object(g.Sphere(0.01), orange)
            contact[str(i)+"A"].set_transform(tf.translation_matrix(pair.p_WCa))
            contact[str(i)+"B"].set_object(g.Sphere(0.01), orange)
            contact[str(i)+"B"].set_transform(tf.translation_matrix(pair.p_WCb))
            contact[str(i)+"normal"].set_object(g.Sphere(0.02), green)
            contact[str(i)+"normal"].set_transform(tf.translation_matrix(info.contact_point()-pair.nhat_BA_W ))
            contact[str(i)+"force"].set_object(g.Sphere(0.02), blue)
            contact[str(i)+"force"].set_transform(tf.translation_matrix(info.contact_point()+info.contact_force()/5000.0 ))
            textoutput.value += f"slip speed:{info.slip_speed():.4f}, separation_speed:{info.separation_speed():.4f}, depth:{pair.depth:.4f},\n"
            textoutput.value += "  point:" + np.array2string(info.contact_point(), formatter={'float': lambda x: "{:5.2f}".format(x)}) + "\n"
            textoutput.value += "  normal:" + np.array2string(-pair.nhat_BA_W, formatter={'float': lambda x: "{:5.2f}".format(x)}) + "\n"
            textoutput.value += "  force:" + np.array2string(info.contact_force(), formatter={'float': lambda x: "{:5.2f}".format(x)}) + "\n"


    lower_limit = [-3, -3, -np.pi/2.0]
    upper_limit = [3, 3, np.pi/2.0]
#    lower_limit += lower_limit
#    upper_limit += upper_limit
    plant.SetPositions(
        plant.GetMyContextFromRoot(context),
#        [0, 0, 0] +
        [1.2, 1.2, 0.0],
    )

    MakeJointSlidersThatPublishOnCallback(
        plant, diagram, context, my_callback, 
        lower_limit=lower_limit, upper_limit=upper_limit)    

contact_inspector(shapeA, shapeB);

# Contact geometry of the foam brick

In [ ]:
def show_brick_contact():
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.01)
    Parser(plant).AddModelFromFile(FindResource("models/061_foam_brick.sdf"))
    plant.Finalize()

    meshcat = ConnectMeshcatVisualizer(
        builder, 
        scene_graph, 
        zmq_url=zmq_url,
        server_args=server_args)

    diagram = builder.Build()
    meshcat.load()

show_brick_contact()

# Point cloud processing with Open3d

I've produced a scene with multiple cameras looking at our favorite YCB mustard bottle.  I've taken the individual point clouds, converted them into Open3d's point cloud format, estimated their normals, merged the point clouds, cropped then point clouds (to get rid of the geometry from the other cameras), then downsampled the point clouds.  (The order is important!)

I've pushed all of the point clouds to meshcat, but with many of them set to not be visible by default.  Use the drop-down menu to turn them on and off, and make sure you understand basically what is happening on each of the steps.

In [ ]:
def point_cloud_processing_example():
    # This just sets up our mustard bottle with three depth cameras positioned around it.
    system = MustardExampleSystem()

    plant = system.GetSubsystemByName("plant")

    # Evaluate the camera output ports to get the images.
    context = system.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)

    v = meshcat.Visualizer(zmq_url=zmq_url)
    v.delete()
    v["/Background"].set_property("visible", False)

    pcd = []
    for i in range(3):
        point_cloud = system.GetOutputPort(f"camera{i}_point_cloud").Eval(context)
        cloud = create_open3d_point_cloud(point_cloud)

        draw_open3d_point_cloud(v[f"pointcloud{i}"], cloud)
        v[f"pointcloud{i}"].set_property("visible", False)

        # Crop to region of interest.
        pcd.append(cloud.crop(
            o3d.geometry.AxisAlignedBoundingBox(min_bound=[-.3, -.3, -.3],
                                                max_bound=[.3, .3, .3])))
        draw_open3d_point_cloud(v[f"pointcloud{i}_cropped"], pcd[i])
        v[f"pointcloud{i}_cropped"].set_property("visible", False)

        pcd[i].estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(
            radius=0.1, max_nn=30))

        camera = plant.GetModelInstanceByName(f"camera{i}")
        body = plant.GetBodyByName("base", camera)
        X_C = plant.EvalBodyPoseInWorld(plant_context, body)
        pcd[i].orient_normals_towards_camera_location(X_C.translation())

    # Merge point clouds.  (Note: You might need something more clever here for
    # noisier point clouds; but this can often work!)
    merged_pcd = pcd[0] + pcd[1] + pcd[2]
    draw_open3d_point_cloud(v["merged"], merged_pcd)

    # Voxelize down-sample.  (Note that the normls still look reasonable)
    down_sampled_pcd = merged_pcd.voxel_down_sample(voxel_size=0.005)
    draw_open3d_point_cloud(v["down_sampled"], down_sampled_pcd, normals_scale=0.01)
    # Let the normals be drawn, only turn off the object...
    v["down_sampled"]["<object>"].set_property("visible", False)

    # If we wanted to show it in the open3d visualizer, we would use...
    # print("Use 'n' to show normals, and '+/-' to change their size.")
    # o3d.visualization.draw_geometries([down_sampled_pcd])


point_cloud_processing_example()

# Estimating normals (and local curvature)

TODO: Add the version from depth images (nearest pixels instead of nearest neighbors), and implement it in DepthImageToPointCloud.

In [ ]:
from ipywidgets import IntSlider

def normal_estimation():
    system = MustardExampleSystem()
    context = system.CreateDefaultContext()

    v = meshcat.Visualizer(zmq_url=zmq_url)
    v.delete()
    v["/Background"].set_property("visible", False)

    point_cloud = system.GetOutputPort("camera0_point_cloud").Eval(context)
    pcd = create_open3d_point_cloud(point_cloud)
    draw_open3d_point_cloud(v["point_cloud"], pcd)

    kdtree = o3d.geometry.KDTreeFlann(pcd)
    pts = np.asarray(pcd.points)

    def my_callback(change):
        query = pts[change.new,:]
        draw_points(v["query"], query, [0.0, 1.0, 0.0], size=0.002)
        (num, indices, distances) = kdtree.search_hybrid_vector_3d(
            query=query, radius=0.1, max_nn=40)

        neighbors = pts[indices, :]
        draw_points(v["neighbors"], neighbors.T, [0.0, 0.0, 1.0], size=0.001)

        pstar = np.mean(neighbors,axis=0)
        prel = neighbors - pstar
        W = np.matmul(prel.T, prel)
        w, V = np.linalg.eigh(W)
        v["least_squares_basis"].set_object(g.triad(0.01))
        v["least_squares_basis"].set_transform(RigidTransform(RotationMatrix(np.fliplr(V)), query).GetAsMatrix4())

    slider = IntSlider(min=0, max=pts.shape[0])
    slider.observe(my_callback, names='value')
    slider.value=4165  # force callback immediately 
    display(slider)

normal_estimation()

# Scoring grasp candidates

In [ ]:
def grasp_candidate_cost(plant_context, cloud, plant, scene_graph, scene_graph_context, adjust_X_G=False, textbox=None, meshcat=None):
    body = plant.GetBodyByName("body")
    X_G = plant.GetFreeBodyPose(plant_context, body)

    # Transform cloud into gripper frame
    X_GW = X_G.inverse()
    pts = np.asarray(cloud.points).T
    p_GC = X_GW.multiply(pts)

    # Crop to a region inside of the finger box.
    crop_min = [-.05, 0.1, -0.00625]
    crop_max = [.05, 0.1125, 0.00625]
    indices = np.all((crop_min[0] <= p_GC[0,:], p_GC[0,:] <= crop_max[0],
                      crop_min[1] <= p_GC[1,:], p_GC[1,:] <= crop_max[1],
                      crop_min[2] <= p_GC[2,:], p_GC[2,:] <= crop_max[2]), 
                     axis=0)

    if meshcat:
        draw_points(meshcat["points"], pts[:, indices], [1., 0, 0], size=0.01)

    if adjust_X_G and np.sum(indices)>0:
        p_GC_x = p_GC[0, indices]
        p_Gcenter_x = (p_GC_x.min() + p_GC_x.max())/2.0
        X_G.set_translation(X_G.translation() + X_G.rotation().multiply([p_Gcenter_x, 0, 0]))
        plant.SetFreeBodyPose(plant_context, body, X_G)
        X_GW = X_G.inverse()

    query_object = scene_graph.get_query_output_port().Eval(scene_graph_context)
    # Check collisions between the gripper and the sink
    if query_object.HasCollisions():
        cost = np.inf
        if textbox:
            textbox.value = "Gripper is colliding with the sink!\n"
            textbox.value += f"cost: {cost}"
        return cost

    # Check collisions between the gripper and the point cloud
    margin = 0.0  # must be smaller than the margin used in the point cloud preprocessing.
    for pt in cloud.points:
        distances = query_object.ComputeSignedDistanceToPoint(pt, threshold=margin)
        if distances:
            cost = np.inf
            if textbox:
                textbox.value = "Gripper is colliding with the point cloud!\n"
                textbox.value += f"cost: {cost}"
            return cost


    n_GC = X_GW.rotation().multiply(np.asarray(cloud.normals)[indices,:].T)

    # Penalize deviation of the gripper from vertical.
    # weight * -dot([0, 0, -1], R_G * [0, 1, 0]) = weight * R_G[2,1]
    cost = 20.0*X_G.rotation().matrix()[2, 1]

    # Reward sum |dot product of normals with gripper x|^2
    cost -= np.sum(n_GC[0,:]**2)

    if textbox:
        textbox.value = f"cost: {cost}\n"
        textbox.value += "normal terms:" + str(n_GC[0,:]**2)
    return cost


def process_point_cloud(diagram, context, cameras, bin_name):
    """A "no frills" version of the example above, that returns the down-sampled point cloud"""
    plant = diagram.GetSubsystemByName("plant")
    plant_context = plant.GetMyContextFromRoot(context)

    # Compute crop box.
    bin_instance = plant.GetModelInstanceByName(bin_name)
    bin_body = plant.GetBodyByName("bin_base", bin_instance)
    X_B = plant.EvalBodyPoseInWorld(plant_context, bin_body)
    margin = 0.001  # only because simulation is perfect!
    a = X_B.multiply([-.22+0.025+margin, -.29+0.025+margin, 0.015+margin])
    b = X_B.multiply([.22-0.1-margin, .29-0.025-margin, 2.0])
    crop_min = np.minimum(a,b)
    crop_max = np.maximum(a,b)

    # Evaluate the camera output ports to get the images.
    merged_pcd = o3d.geometry.PointCloud()
    for c in cameras:
        point_cloud = diagram.GetOutputPort(f"{c}_point_cloud").Eval(context)
        pcd = create_open3d_point_cloud(point_cloud)

        # Crop to region of interest.
        pcd = pcd.crop(
            o3d.geometry.AxisAlignedBoundingBox(min_bound=crop_min,
                                                max_bound=crop_max))    

        pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(
            radius=0.1, max_nn=30))

        camera = plant.GetModelInstanceByName(c)
        body = plant.GetBodyByName("base", camera)
        X_C = plant.EvalBodyPoseInWorld(plant_context, body)
        pcd.orient_normals_towards_camera_location(X_C.translation())
        
        # Merge point clouds.
        merged_pcd += pcd

    # Voxelize down-sample.  (Note that the normals still look reasonable)
    return merged_pcd.voxel_down_sample(voxel_size=0.005)

def make_environment_model(directive=None, draw=False, rng=None, num_ycb_objects=0, bin_name="bin0"):
    # Make one model of the environment, but the robot only gets to see the sensor outputs.
    if not directive:
        directive = FindResource("models/two_bins_w_cameras.yaml")

    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0005)
    parser = Parser(plant)
    parser.package_map().PopulateFromFolder(FindResource(""))
    ProcessModelDirectives(LoadModelDirectives(directive), plant, parser)

    for i in range(num_ycb_objects):
        object_num = rng.integers(len(ycb))
        sdf = FindResourceOrThrow("drake/manipulation/models/ycb/sdf/" + ycb[object_num][1])
        parser.AddModelFromFile(sdf, f"object{i}")

    plant.Finalize()
    AddRgbdSensors(builder, plant, scene_graph)

    if draw:
        meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, prefix="environment")

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()

    if num_ycb_objects > 0:
        generator = RandomGenerator(rng.integers(1000))  # this is for c++
        plant_context = plant.GetMyContextFromRoot(context)
        bin_instance = plant.GetModelInstanceByName(bin_name)
        bin_body = plant.GetBodyByName("bin_base", bin_instance)
        X_B = plant.EvalBodyPoseInWorld(plant_context, bin_body)
        z = 0.1
        for body_index in plant.GetFloatingBaseBodies():
            tf = RigidTransform(
                    UniformlyRandomRotationMatrix(generator),  
                    [rng.uniform(-.15,.15), rng.uniform(-.2, .2), z])
            plant.SetFreeBodyPose(plant_context, 
                                plant.get_body(body_index),
                                X_B.multiply(tf))
            z += 0.1

        simulator = Simulator(diagram, context)
        simulator.AdvanceTo(1.0 if running_as_notebook else 0.1)
    elif draw:
        meshcat.load()
        diagram.Publish(context)


    return diagram, context

def grasp_score_inspector():
    v = meshcat.Visualizer(zmq_url=zmq_url)
    v.delete()
    environment, environment_context = make_environment_model(directive = FindResource("models/clutter_mustard.yaml"))

    # Another diagram for the objects the robot "knows about": gripper, cameras, bins.  Think of this as the model in the robot's head.
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    parser = Parser(plant)
    parser.package_map().PopulateFromFolder(FindResource(""))
    ProcessModelDirectives(LoadModelDirectives(FindResource("models/clutter_planning.yaml")), plant, parser)
    plant.Finalize()
    
    v = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, prefix="planning")
    v.load()
    diagram = builder.Build()
    context = diagram.CreateDefaultContext()

    cloud = process_point_cloud(environment, environment_context, ["camera0", "camera1", "camera2"], "bin0")
    draw_open3d_point_cloud(v.vis["cloud"], cloud, size=0.003)

    textbox = Textarea(description="cost info", layout={'width': '800px', 'height': '100px'})
    display(textbox)

    lower_limit = [-np.pi, -np.pi/4., -np.pi/4., -1, -1, 0]
    upper_limit = [0, np.pi/4., np.pi/4., 1, 1, 1]
    sliders = MakeJointSlidersThatPublishOnCallback(
        plant, diagram, context, my_callback=partial(
            grasp_candidate_cost, cloud=cloud, plant=plant, scene_graph=scene_graph,
            scene_graph_context=scene_graph.GetMyContextFromRoot(context),
            textbox=textbox, meshcat=v.vis["cost"]),
        lower_limit=lower_limit, upper_limit=upper_limit)
    sliders[0].value = -np.pi/2.0
    sliders[3].value = -0.05
    sliders[4].value = -.5
    sliders[5].value = 0.25

grasp_score_inspector()

# Generating grasp candidates

In [ ]:

def generate_grasp_candidate_antipodal(plant_context, cloud, plant, scene_graph, scene_graph_context, rng, meshcat=None):
    """
    Picks a random point in the cloud, and aligns the robot finger with the normal of that pixel. 
    The rotation around the normal axis is drawn from a uniform distribution over [min_roll, max_roll].
    """
    body = plant.GetBodyByName("body")

    index = rng.integers(0,len(cloud.points)-1)

    # Use S for sample point/frame.
    p_WS = np.asarray(cloud.points[index])
    n_WS = np.asarray(cloud.normals[index])

    if meshcat:
        vertices = np.empty((3,2))
        vertices[:, 0] = p_WS
        vertices[:, 1] = p_WS + 0.05*n_WS
        meshcat.set_object(g.LineSegments(g.PointsGeometry(vertices),
                            g.MeshBasicMaterial(color=0xff0000)))

    assert np.isclose(np.linalg.norm(n_WS), 1.0)

    Gx = n_WS # gripper x axis aligns with normal
    # make orthonormal y axis, aligned with world down
    y = np.array([0.0, 0.0, -1.0])
    if np.abs(np.dot(y,Gx)) < 1e-6:
        # normal was pointing straight down.  reject this sample.
        return None

    Gy = y - np.dot(y,Gx)*Gx
    Gz = np.cross(Gx, Gy)
    R_WG = RotationMatrix(np.vstack((Gx, Gy, Gz)).T)
    p_GS_G = [0.054 - 0.01, 0.10625, 0]

    # Try orientations from the center out
    min_roll=-np.pi/3.0
    max_roll=np.pi/3.0
    alpha = np.array([0.5, 0.65, 0.35, 0.8, 0.2, 1.0, 0.0])
    for theta in (min_roll + (max_roll - min_roll)*alpha):
        # Rotate the object in the hand by a random rotation (around the normal).
        R_WG2 = R_WG.multiply(RotationMatrix.MakeXRotation(theta))

        # Use G for gripper frame.
        p_SG_W = - R_WG2.multiply(p_GS_G)
        p_WG = p_WS + p_SG_W 

        X_G = RigidTransform(R_WG2, p_WG)
        plant.SetFreeBodyPose(plant_context, body, X_G)
        cost = grasp_candidate_cost(plant_context, cloud, plant, scene_graph, scene_graph_context, adjust_X_G=True, meshcat=meshcat)
        X_G = plant.GetFreeBodyPose(plant_context, body)
        if np.isfinite(cost):
            return cost, X_G

        #draw_grasp_candidate(X_G, f"collision/{theta:.1f}")

    return np.inf, None

def draw_grasp_candidate(X_G, prefix='gripper', draw_frames=True):
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    parser = Parser(plant)
    parser.package_map().Add("wsg_50_description", os.path.dirname(FindResourceOrThrow("drake/manipulation/models/wsg_50_description/package.xml")))
    gripper = parser.AddModelFromFile(FindResource(
        "models/schunk_wsg_50_welded_fingers.sdf"), "gripper")
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("body"), X_G)
    plant.Finalize()
    
    frames_to_draw = {"gripper": {"body"}} if draw_frames else {}
    meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, prefix=prefix, delete_prefix_on_load=False, frames_to_draw=frames_to_draw)
    diagram = builder.Build()
    context = diagram.CreateDefaultContext()

    meshcat.load()
    diagram.Publish(context)

def sample_grasps_example():
    v = meshcat.Visualizer(zmq_url=zmq_url)
    v.delete()
    rng = np.random.default_rng()

    environment, environment_context = make_environment_model(rng=rng, num_ycb_objects=5, draw=False)

    # Another diagram for the objects the robot "knows about": gripper, cameras, bins.  Think of this as the model in the robot's head.
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    parser = Parser(plant)
    parser.package_map().PopulateFromFolder(FindResource(""))
    ProcessModelDirectives(LoadModelDirectives(FindResource("models/clutter_planning.yaml")), plant, parser)
    plant.Finalize()
    
    v = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, prefix="planning")
    v.load()
    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    diagram.Publish(context)
    v.vis["planning/10"].set_property('visible', False)  # Hide this particular gripper

    cloud = process_point_cloud(environment, environment_context, ["camera0", "camera1", "camera2"], "bin0")
    draw_open3d_point_cloud(v.vis["cloud"], cloud, size=0.003)

    plant_context = plant.GetMyContextFromRoot(context)
    scene_graph_context = scene_graph.GetMyContextFromRoot(context)

    costs = []
    X_Gs = []
    for i in range(100):
        cost, X_G = generate_grasp_candidate_antipodal(plant_context, cloud, plant, scene_graph, scene_graph_context, rng)#, meshcat=v.vis["sample"])
        if np.isfinite(cost):
            costs.append(cost)
            X_Gs.append(X_G)

    indices = np.asarray(costs).argsort()[:5]
    for i in indices:
        draw_grasp_candidate(X_Gs[i], prefix=f"{i}th best", draw_frames=False)


sample_grasps_example()